# Run using AWS_BedRock_Anthropic_Claude_V2 Notebook template (Python 3.9 with numpy == 1.24.0 and refractml==1.0.3, Medium Container)


In [1]:
!pip install openai==0.28.1

     |████████████████████████████████| 81kB 3.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/00/e5/f12a80907d0884e6dff9c16d0c0114d81b8cd07dc3ae54c5e962cc83037e/tqdm-4.66.1-py3-none-any.whl
     |████████████████████████████████| 1.1MB 21.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf7c3d9/certifi-2023.7.22-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/98/69/5d8751b4b670d623aa7a47bef061d69c279e9f922f6705147983aa76c3ce/charset_normalizer-3.3.2-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fc/34/3030de6f1370931b9dbb4dad48f6ab1015ab1d32447850b9fc94e60097be/idna-3.4-py3-none-any.whl
     |████████████████████████████████| 112kB 54.7MB/

In [2]:
AZURE_OPENAI_KEY=openai_key
AZURE_OPENAI_ENDPOINT="https://gear.openai.azure.com/"

In [3]:
import os
import openai

openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-07-01-preview' # this might change in the future
deployment_name='gpt-35-turbo' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

In [44]:
message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":"who is President of India ?"},{"role":"assistant","content":"As of September 2021, the Prime Minister of India is Narendra Modi."}]

In [45]:
completion = openai.ChatCompletion.create(
  engine="gpt-35-turbo",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://gear.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview
DEBUG:openai:api_version=2023-07-01-preview data='{"messages": [{"role": "system", "content": "You are an AI assistant that helps people find information."}, {"role": "user", "content": "who is President of India ?"}, {"role": "assistant", "content": "As of September 2021, the Prime Minister of India is Narendra Modi."}], "temperature": 0.7, "max_tokens": 800, "top_p": 0.95, "frequency_penalty": 0, "presence_penalty": 0, "stop": null}' message='Post details'
DEBUG:urllib3.connectionpool:https://gear.openai.azure.com:443 "POST //openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview HTTP/1.1" 200 870
DEBUG:openai:message='OpenAI API response' path=https://gear.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview processing_ms=None request_id=8df5d6

In [46]:
completion

<OpenAIObject chat.completion id=chatcmpl-8L7I6Lsw60DUeuR2757NnsbKBdrMA at 0x7ff66877ad60> JSON: {
  "id": "chatcmpl-8L7I6Lsw60DUeuR2757NnsbKBdrMA",
  "object": "chat.completion",
  "created": 1700043606,
  "model": "gpt-35-turbo",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "self_harm": {
          "filtered": false,
          "severity": "safe"
        },
        "sexual": {
          "filtered": false,
          "severity": "safe"
        },
        "violence": {
          "filtered": false,
          "severity": "safe"
        }
      }
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Apologies for the confusion in my previous response. As of September 2021, the President of India is Ram Nath Kovind."
      },
      "content_filter_results":

In [47]:
completion['choices'][0]['message']['content'].replace('\n', '').replace(' .', '.').strip()

'Apologies for the confusion in my previous response. As of September 2021, the President of India is Ram Nath Kovind.'

In [17]:
from refractml import *
from refractml.constants import MLModelFlavours

# new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

In [52]:
# Model Registration
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        import openai
        
        openai.api_key = self.AZURE_OPENAI_KEY
        openai.api_base = "https://gear.openai.azure.com/" #self.AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
        openai.api_type = 'azure'
        openai.api_version = '2023-07-01-preview' # this might change in the future
        deployment_name='gpt-35-turbo'
    
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[Input data, ..]. It could be:
               A List Mapping of All Value Can Be one of : 
                   - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                   - List[numpy.array(), numpy.array(), ...]
                   - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                   - List[ SingleSample, SingleSample]
                   
        :return: (n_inputs, payload's)
        
        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 
    
    def pre_processing_fn(self,payload):
        # All preprocessing step must occur in this section
        # Takes Single Sample -> Returns Single Sample
        
        # Not Doing Any Preprocessing Hence Returned payload
        #print("payload is ", payload)
        return payload

    def prediction_fn(self,
                      model: Any,
                      pre_processed_input 
                      ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        #self.message_text 
        conversation=[{"role": "system", "content": "You are a helpful assistant."}]
        conversation.append({"role": "user", "content": pre_processed_input})
        #print (conversation)
        completion = openai.ChatCompletion.create(
            engine="gpt-35-turbo",
            messages = conversation,
            temperature=0.7,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
        
        #print (completion)
        return completion['choices'][0]['message']['content'].replace('\n', '').replace(' .', '.').strip()


    class Meta:    
        # List of Callables() can be attached For Calling After AnSd Before Scoring
        def __init__(self):
            self.name = "Pre Hooked Me !"
            self.pre_call_hooks.append(self.print_)
        def print_(self):
            print(self.name)
        pre_call_hooks = []
        post_call_hooks = []

In [53]:
prompt = "Who is the Defence Minister of India"
import requests
req = requests.Request()
req.json = {"payload":prompt}
req.json

{'payload': 'Who is the Defence Minister of India'}

In [54]:
score_ = ScoreTemplateExample()
model_predictions = score_.score(None, req, dry_run=True)

Pre Hooked Me !


,VALIDATION,COMPONENT,PASSED,SKIPPED
0,"Return Type Must Be Tuple (n_input, payloads)",request_processing_fn,True,False
1,"Tuple Must Be of length Two (n_input, payloads)",request_processing_fn,True,False
2,"* if n_input > 1 payload type must be List (n_input, [np.ndarray, tf.Tensor, etc])",request_processing_fn,True,False


DEBUG:openai:message='Request to OpenAI API' method=post path=https://gear.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview
DEBUG:openai:api_version=2023-07-01-preview data='{"messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Who is the Defence Minister of India"}], "temperature": 0.7, "max_tokens": 800, "top_p": 0.95, "frequency_penalty": 0, "presence_penalty": 0, "stop": null}' message='Post details'
DEBUG:urllib3.util.retry:Converted retries value: 2 -> Retry(total=2, connect=None, read=None, redirect=None, status=None)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gear.openai.azure.com:443


Fields Marked Asterisk (*) Can Be Validated On Proper Input 



DEBUG:urllib3.connectionpool:https://gear.openai.azure.com:443 "POST //openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview HTTP/1.1" 200 820
DEBUG:openai:message='OpenAI API response' path=https://gear.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview processing_ms=None request_id=50418006-b5d0-46ea-b89f-43b1c65908ec response_code=200


In [55]:
print(model_predictions[0].score_response)

As of October 2021, the Defence Minister of India is Rajnath Singh.


In [56]:
register_model(None,
               ScoreTemplateExample,
               "Azure_OpenAI_GPT35_Turbo",
               "Azure_OpenAI_GPT35_Turbo",
               MLModelFlavours.pytorch,
               init_script="pip install openai==0.28.1",
              )

INFO:root:PASS:: Mandatory Validation :: dict_keys(['name', 'description', 'flavour', 'scoring_func'])
INFO:root:PASS:: AlphaNumeric Validation :: dict_keys(['name'])
INFO:root:PASS:: Validation/IfPresentTypeCheck :: dict_keys(['schema', 'metadata_info', 'tags'])
INFO:root:PASS:: Validation/IfPresentSubfieldMustExist :: dict_keys(['kyd'])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "POST /registry/api/v1/ml-model/register HTTP/1.1" 200 2041
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/32f760e2-9efb-49fb-9559-1cb279d717c9 HTTP/1.1" 200 2034
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): mosaic-ai-backend:5000
DEBUG:urllib3.connectionpool:http://mosaic-ai-backend:5000 "GET /registry/api/v1/ml-model/32f760e2-9efb-49fb-9559-1cb279d717c9 HTTP/1.1"